## Chapter 5: .NET Aspire Components

In the previous chapter, you learned how to model your application in the AppHost, adding resources like PostgreSQL and RabbitMQ and connecting them to your services. But we only scratched the surface of how your services actually *use* those resources. That’s where **.NET Aspire Components** come in.

Components are NuGet packages that provide a streamlined, opinionated way to integrate popular services (databases, message brokers, cloud services) into your .NET applications. They handle the boilerplate of registering clients, setting up health checks, enabling telemetry, and configuring resilience—all with minimal code.

In this chapter, we’ll explore what components are, how they work, and how to use them to connect to PostgreSQL and Redis. By the end, you’ll be able to add real data persistence and caching to your Aspire applications.

---

### 5.1 What Are .NET Aspire Components?

An **Aspire component** is a NuGet package that:

- Registers a client library for a specific service (e.g., `Npgsql` for PostgreSQL, `StackExchange.Redis` for Redis) in the dependency injection container.
- Automatically reads the connection string from configuration (which, in an Aspire app, is injected by the AppHost via `WithReference`).
- Adds health checks specific to that service (e.g., checking if the database is reachable).
- Enables OpenTelemetry instrumentation for that service (traces, metrics, logs).
- Optionally adds resilience policies (e.g., retries for transient database errors).

All of this is done with a single line of code, like `builder.AddNpgsqlDataSource("mydb")`. This aligns with Aspire’s philosophy of “batteries‑included but replaceable”—you get production‑ready defaults, but you can customize every aspect.

#### 5.1.1 The “Connection String” Abstraction

One of the key ideas in Aspire is that connection strings are **injected** by the AppHost, not hard‑coded in the service. When you call `.WithReference(postgres)` on a project resource, the AppHost adds an environment variable (or configuration key) named `ConnectionStrings__postgres` with the correct connection string. The component then reads this key automatically.

This means your service code never needs to know where the database is running; it just uses a logical name (`"postgres"`) that matches the resource name in the AppHost.

---

### 5.2 How Components Work Under the Hood

Let’s look at a typical component, `Aspire.Npgsql`. When you call `builder.AddNpgsqlDataSource("postgres")` in your service’s `Program.cs`, the component:

1. Fetches the connection string from configuration using the name `"postgres"` (the key `ConnectionStrings:postgres`).
2. Registers `NpgsqlDataSource` (a pooled connection factory) in DI as a singleton.
3. Adds a health check that tries to open a connection to the database.
4. Adds OpenTelemetry instrumentation: traces for SQL commands, metrics for connection pool statistics.
5. Optionally adds resilience (if configured) using Polly.

All of this is implemented via standard .NET extension methods. You can see the source code or decompile to understand exactly what’s registered.

#### 5.2.1 Configuration and Options

Components are highly configurable. Each component has an options class (e.g., `NpgsqlSettings`) that you can set via code or configuration. For example:

```csharp
builder.AddNpgsqlDataSource("postgres", settings =>
{
    settings.HealthChecks = true;        // enable/disable health checks
    settings.Tracing = true;              // enable/disable tracing
    settings.Metrics = true;               // enable/disable metrics
});
```

You can also configure via `appsettings.json` using the standard .NET options pattern:

```json
{
  "Aspire": {
    "Npgsql": {
      "Postgres": {
        "HealthChecks": true,
        "Tracing": true
      }
    }
  }
}
```

This flexibility lets you tailor the component to your needs.

---

### 5.3 Working with Databases: PostgreSQL

Now let’s add a PostgreSQL database to our `MyAspireApp` solution and use it in the API service.

#### 5.3.1 Adding PostgreSQL to the AppHost

First, we need to add the PostgreSQL hosting package to the AppHost project:

```bash
cd MyAspireApp.AppHost
dotnet add package Aspire.Hosting.PostgreSQL
```

Now update `Program.cs` in the AppHost to include a PostgreSQL resource and reference it from the API service:

```csharp
using Aspire.Hosting;

var builder = DistributedApplication.CreateBuilder(args);

// Add PostgreSQL container
var postgres = builder.AddPostgres("postgres")
    .WithDataVolume();  // Persists data across restarts

// Add a database inside the PostgreSQL instance
var productsDb = postgres.AddDatabase("productsdb");

var apiService = builder.AddProject<Projects.MyAspireApp_ApiService>("apiservice")
    .WithReference(productsDb)   // This injects the connection string for "productsdb"
    .WithEnvironment("ASPNETCORE_ENVIRONMENT", "Development"); // optional

var webfrontend = builder.AddProject<Projects.MyAspireApp_Web>("webfrontend")
    .WithReference(apiService);

builder.Build().Run();
```

**Explanation**:
- `AddPostgres("postgres")` adds a PostgreSQL container resource with logical name `"postgres"`. It uses the official PostgreSQL image.
- `WithDataVolume()` creates a Docker volume to persist database files; otherwise data would be lost when the container stops.
- `AddDatabase("productsdb")` creates a named database inside the PostgreSQL instance. This is a child resource that produces a connection string scoped to that specific database.
- In the API service, `WithReference(productsDb)` injects the connection string for `productsdb` as `ConnectionStrings__productsdb`.

#### 5.3.2 Adding the PostgreSQL Component to the API Service

Now go to the API service project and add the component NuGet package:

```bash
cd ../MyAspireApp.ApiService
dotnet add package Aspire.Npgsql
```

In `Program.cs` of the API service, add the PostgreSQL client:

```csharp
using MyAspireApp.ServiceDefaults;

var builder = WebApplication.CreateBuilder(args);

builder.AddServiceDefaults();

// Add PostgreSQL client
builder.AddNpgsqlDataSource("productsdb");

// ... rest of your service configuration
```

The string `"productsdb"` matches the resource name we used in `WithReference`. The component will look for `ConnectionStrings:productsdb` in configuration.

#### 5.3.3 Using the Database in an Endpoint

Now let’s create a simple products API that uses the database. We’ll need Entity Framework Core to map objects to tables, but for simplicity we’ll use raw SQL with `NpgsqlDataSource`. In a real application, you’d likely use EF Core, and Aspire provides a component for that too (`Aspire.Npgsql.EntityFrameworkCore.PostgreSQL`). We’ll stick with raw `Npgsql` for this example.

First, install the Dapper micro-ORM for easier data access (optional but convenient):

```bash
dotnet add package Dapper
```

Create a `Product` class:

```csharp
namespace MyAspireApp.ApiService;

public class Product
{
    public int Id { get; set; }
    public string Name { get; set; } = string.Empty;
    public decimal Price { get; set; }
}
```

Now add endpoints to `Program.cs`:

```csharp
app.MapGet("/products", async (NpgsqlDataSource dataSource) =>
{
    using var connection = await dataSource.OpenConnectionAsync();
    var products = await connection.QueryAsync<Product>("SELECT * FROM Products");
    return Results.Ok(products);
});

app.MapPost("/products", async (Product product, NpgsqlDataSource dataSource) =>
{
    using var connection = await dataSource.OpenConnectionAsync();
    var id = await connection.ExecuteScalarAsync<int>(
        "INSERT INTO Products (Name, Price) VALUES (@Name, @Price) RETURNING Id", product);
    product.Id = id;
    return Results.Created($"/products/{id}", product);
});
```

But before these endpoints work, we need to create the `Products` table. We can do that on startup by adding a database initializer. For simplicity, let’s add a simple check in the API’s startup:

```csharp
// After app.Build()
using (var scope = app.Services.CreateScope())
{
    var dataSource = scope.ServiceProvider.GetRequiredService<NpgsqlDataSource>();
    using var connection = await dataSource.OpenConnectionAsync();
    await connection.ExecuteAsync(@"
        CREATE TABLE IF NOT EXISTS Products (
            Id SERIAL PRIMARY KEY,
            Name TEXT NOT NULL,
            Price DECIMAL NOT NULL
        )");
}
```

This ensures the table exists when the API starts.

#### 5.3.4 Observing Telemetry

Run the AppHost. In the dashboard, you’ll now see traces that include database spans. When you call the `/products` endpoint, you’ll see a span for the SQL query, including its duration and any parameters. This is automatically added by the component’s tracing instrumentation.

Health checks are also automatically added. The API’s `/health` endpoint will now include a check named `productsdb` that verifies database connectivity. You can see this in the dashboard or by hitting `/health` directly.

---

### 5.4 Working with Redis for Caching

Redis is a popular in‑memory data store often used for caching. Let’s add Redis to our AppHost and use it in the API service to cache product data.

#### 5.4.1 Adding Redis to the AppHost

Add the Redis hosting package to the AppHost:

```bash
cd ../MyAspireApp.AppHost
dotnet add package Aspire.Hosting.Redis
```

Update `Program.cs` to include Redis:

```csharp
var redis = builder.AddRedis("cache")
    .WithDataVolume();  // optional, if you want persistence

apiService.WithReference(redis);  // inject connection string for Redis
```

Now the API service will have a connection string named `ConnectionStrings__cache`.

#### 5.4.2 Adding the Redis Component to the API Service

In the API service project, add the Redis component:

```bash
cd ../MyAspireApp.ApiService
dotnet add package Aspire.StackExchange.Redis
```

In `Program.cs`, add the Redis client:

```csharp
builder.AddRedisClient("cache");
```

This registers `IConnectionMultiplexer` in DI, which can be used to interact with Redis.

#### 5.4.3 Using Redis for Caching

Modify the `GET /products` endpoint to cache the list of products for 30 seconds:

```csharp
app.MapGet("/products", async (NpgsqlDataSource dataSource, IConnectionMultiplexer redis) =>
{
    var db = redis.GetDatabase();
    var cached = await db.StringGetAsync("products");
    if (cached.HasValue)
    {
        return Results.Ok(JsonSerializer.Deserialize<List<Product>>(cached!));
    }

    using var connection = await dataSource.OpenConnectionAsync();
    var products = (await connection.QueryAsync<Product>("SELECT * FROM Products")).ToList();

    await db.StringSetAsync("products", JsonSerializer.Serialize(products), TimeSpan.FromSeconds(30));

    return Results.Ok(products);
});
```

This simple caching strategy reduces database load. The Redis component automatically enables health checks and telemetry for Redis, so you’ll see Redis calls in the traces.

---

### 5.5 Working with Azure Services (Overview)

Aspire also includes components for Azure services like Blob Storage, Service Bus, and Azure Key Vault. These components work similarly but often include support for emulators (like Azurite for Storage) so you can develop locally without an Azure subscription.

For example, to use Azure Blob Storage, you’d add `Aspire.Azure.Storage.Blobs` to your project and in the AppHost:

```csharp
var blobs = builder.AddAzureStorage("storage")
    .RunAsEmulator()  // runs Azurite container
    .AddBlobs("products");
```

Then in your service, call `builder.AddAzureBlobClient("products")`. The component will handle connection to the emulator or real Azure service based on configuration.

We’ll cover Azure components in more detail in a later chapter.

---

### 5.6 Configuring Component Behavior

Components are highly configurable. You can set options via code or configuration. Let’s look at a few examples.

#### 5.6.1 Disabling Health Checks for a Specific Component

If you don’t want health checks for a particular Redis instance, you can disable them:

```csharp
builder.AddRedisClient("cache", settings =>
{
    settings.HealthChecks = false;
});
```

#### 5.6.2 Customizing Connection String Retrieval

By default, components look for `ConnectionStrings:{name}`. You can override this by providing a connection string directly in code, though that’s not recommended because it ties the service to a specific location.

#### 5.6.3 Configuring Resilience

Some components, like `Aspire.Npgsql`, support resilience policies. You can configure retry options via the settings:

```csharp
builder.AddNpgsqlDataSource("productsdb", settings =>
{
    settings.Retry.MaxRetryCount = 5;
    settings.Retry.MaxRetryDelay = TimeSpan.FromSeconds(5);
});
```

This uses Polly behind the scenes.

#### 5.6.4 Using Configuration Files

You can also put component settings in `appsettings.json`:

```json
{
  "Aspire": {
    "Npgsql": {
      "Productsdb": {
        "HealthChecks": true,
        "Tracing": true,
        "Metrics": true,
        "Retry": {
          "MaxRetryCount": 5
        }
      }
    }
  }
}
```

The section name `Productsdb` matches the resource name. This keeps configuration out of code.

---

### 5.7 Hands‑on: Adding PostgreSQL and Redis to Your Application

Now it’s your turn. Follow these steps to integrate PostgreSQL and Redis into your existing `MyAspireApp` solution.

**Step 1: Add the hosting packages to AppHost**

```bash
cd MyAspireApp.AppHost
dotnet add package Aspire.Hosting.PostgreSQL
dotnet add package Aspire.Hosting.Redis
```

**Step 2: Update AppHost Program.cs** as shown in sections 5.3.1 and 5.4.1.

**Step 3: Add component packages to ApiService**

```bash
cd ../MyAspireApp.ApiService
dotnet add package Aspire.Npgsql
dotnet add package Aspire.StackExchange.Redis
dotnet add package Dapper
```

**Step 4: Modify ApiService Program.cs** to add the components, create the table on startup, and implement the cached products endpoint as shown.

**Step 5: Run the AppHost** and test the `/products` endpoint (you may need to create a few products via POST first). Use the dashboard to observe traces and health checks.

**Step 6: Experiment** – stop the PostgreSQL container (via Docker) and see how the health check fails and the API returns errors (unless you have caching). Then restart it and see recovery.

---

### 5.8 Summary

In this chapter, you learned about .NET Aspire Components—NuGet packages that simplify connecting to external services. Key takeaways:

- Components automatically register clients, health checks, telemetry, and resilience.
- They read connection strings injected by the AppHost via `WithReference`.
- You can configure them via code or configuration files.
- We added PostgreSQL and Redis to our application, demonstrating real data persistence and caching.
- Components make it easy to follow best practices with minimal code.

In the next chapter, we’ll explore **Messaging and Event‑Driven Architecture** with Aspire. You’ll learn how to use components for RabbitMQ and Azure Service Bus to build decoupled, scalable systems. We’ll also dive into more advanced patterns like competing consumers and publish/subscribe.

---

**Exercises**

1. Add an Entity Framework Core component (`Aspire.Npgsql.EntityFrameworkCore.PostgreSQL`) to the API service and rewrite the products endpoints using EF Core. Compare the code complexity.
2. Add a Redis cache to the worker service and cache processed message IDs to avoid duplicate processing.
3. Explore the options of the PostgreSQL component by setting `HealthChecks` to false and observing the health endpoint.

In Chapter 6, we’ll build on this foundation by adding message queues and implementing an event‑driven order processing workflow.